In [ ]:

%pip uninstall -y unsloth unsloth_zoo trl
%pip install --upgrade --force-reinstall --no-cache-dir "trl>=0.14,<0.17" unsloth unsloth_zoo

import IPython, sys
print("Installed. Python:", sys.version)
print("Please now go to: Runtime -> Restart runtime, then re-run the next cell.")

Found existing installation: unsloth 2025.10.8
Uninstalling unsloth-2025.10.8:
  Successfully uninstalled unsloth-2025.10.8
Found existing installation: unsloth_zoo 2025.10.9
Uninstalling unsloth_zoo-2025.10.9:
  Successfully uninstalled unsloth_zoo-2025.10.9
Found existing installation: trl 0.16.1
Uninstalling trl-0.16.1:
  Successfully uninstalled trl-0.16.1
Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of unsloth-zoo to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 176.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.

In [ ]:
import sys
print("Python:", sys.version)

import unsloth, transformers, trl
print("unsloth =", getattr(unsloth, "__version__", "?"))
print("transformers =", transformers.__version__)
print("trl =", trl.__version__)

import importlib
rl = importlib.import_module("unsloth.models.rl")
print("has align_logprobs_with_mask:", "align_logprobs_with_mask" in rl.RL_REPLACEMENTS)

Python: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:45:31) [GCC 13.3.0]
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!
unsloth = 2025.10.8
transformers = 4.56.2
trl = 0.16.1
has align_logprobs_with_mask: False


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024  # maximum sequence length for inputs
dtype = None  # auto-select optimal dtype based on gpu capability
load_in_4bit = True  # enable 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # base model for fine-tuning
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.179 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import load_dataset

full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(20000))
validation_dataset = shuffled_dataset.select(range(20000, 22000))

In [ ]:
training_prompt = """You are a great and STRICT mathematician. Your task is to see and judge if the proposed Solution solves the Question correctly. You are to respond with EXACTLY one word on a single line. You are to respond with either 'True' if the solution is correct or 'False' if the solution is NOT correct. DO NOT ADD ANY PUNCTUATION OR ANY OTHER TEXT OTHER THAN 'True' or 'False'.

Question:
{}

Solution:
{}

Output:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # lora rank determines adapter capacity, 32 provides good balance
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  # attention and mlp layers to adapt
    lora_alpha = 64,  # scaling factor, typically 2x rank for balanced adaptation
    lora_dropout = 0.05,  # low dropout to maintain performance while regularizing
    bias = "none",  # no bias terms trained to reduce parameters
    use_gradient_checkpointing = "unsloth",  # memory efficient training
    random_state = 42,  # reproducibility seed
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,  # small batch size to fit in h100 memory
        num_train_epochs = 3,  # full passes through training data
        gradient_accumulation_steps = 8,  # effective batch size is 2*8=16 per device
        warmup_steps = 100,  # linear warmup for stable training start
        learning_rate = 2e-4,  # standard lora learning rate
        fp16 = not torch.cuda.is_bf16_supported(),  # use fp16 if bf16 not available
        bf16 = torch.cuda.is_bf16_supported(),  # prefer bf16 for better numerical stability
        logging_steps = 20,  # log metrics every 20 steps
        optim = "adamw_8bit",  # memory efficient optimizer
        weight_decay = 0.05,  # l2 regularization to prevent overfitting
        lr_scheduler_type = "cosine",  # cosine annealing for smooth decay
        seed = 42,  # random seed for reproducibility
        output_dir = "outputs",  # directory for checkpoints
        report_to = "none",  # disable external logging services
        save_strategy = "epoch",  # save model after each epoch
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=228):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3 | Total steps = 3,750
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,1.389500
40,0.758300
60,0.665100
80,0.626200
100,0.619700
120,0.641600
140,0.615400
160,0.617900
180,0.639200
200,0.608300


TrainOutput(global_step=3750, training_loss=0.4264144443511963, metrics={'train_runtime': 9577.4127, 'train_samples_per_second': 6.265, 'train_steps_per_second': 0.392, 'total_flos': 1.0790728835110502e+18, 'train_loss': 0.4264144443511963})

In [ ]:
FastLanguageModel.for_inference(model)

inference_prompt = """You are a great and STRICT mathematician. Your task is to see and judge if the proposed Solution solves the Question correctly. You are to respond with EXACTLY one word on a single line. You are to respond with either 'True' if the solution is correct or 'False' if the solution is NOT correct. DO NOT ADD ANY PUNCTUATION OR ANY OTHER TEXT OTHER THAN 'True' or 'False'.

Question:
{}

Solution:
{}

Output:
"""

example = validation_dataset[10]
question = example["question"]
solution = example["solution"]

inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1, do_sample = False, temperature = 0.0, pad_token_id = tokenizer.eos_token_id, eos_token_id = tokenizer.eos_token_id, use_cache = True)
response = tokenizer.batch_decode(outputs)

print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
How many unordered pairs of prime numbers have a sum of 40?

#### SOLUTION ####
To compute all the prime numbers with sum of 40, we can enumerate all pairs of primes where sum = 40. 
One such instance can be solved by enumeration as follows:
<llm-code>
import itertools

answer = []
# Max sum of two primes below 40 is 39, so limiting ourselves to that
# Prevents us from examining the same pairing (i.e., [3,39]) again
for p1, p2 in itertools.product(range(2,20), range(2,39)):
    if p1 + p2 == 40:
        answer.append((p1, p2))

print(len(answer))
</llm-code>
<llm-code-output>
18
</llm-code-output>
Hence, there are \boxed{18} unordered pairs of prime numbers that have a sum of 40.

#### MODEL'S PREDICTION ####
False

#### CORRECT ANSWER ####
False


In [ ]:
import pandas as pd
from tqdm import tqdm

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    if 'true' in output_part.lower():
        return True
    return False

for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 1, do_sample = False, temperature = 0.0, pad_token_id = tokenizer.eos_token_id, eos_token_id = tokenizer.eos_token_id, use_cache = True)
    response_text = tokenizer.batch_decode(outputs)[0]

    prediction = parse_output(response_text)
    predictions.append(prediction)

submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 11%|█         | 1063/10000 [01:44<14:22, 10.37it/s]Unsloth: Input IDs of shape torch.Size([1, 1067]) with length 1067 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 13%|█▎        | 1274/10000 [02:05<14:13, 10.22it/s]Unsloth: Input IDs of shape torch.Size([1, 1234]) with length 1234 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [02:34<14:06,  9.96it/s]Unsloth: Input IDs of shape torch.Size([1, 1076]) with length 1076 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1590/10000 [02:36<13:59, 10.01it/s]Unsloth: Input IDs of shape torch.Size([1, 1047]) with length 1047 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2328/10000 


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.
